In [1]:
import re
import os
import fitz
from bs4 import BeautifulSoup
from semantic_text_splitter import TextSplitter
import sys, numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings


c:\Users\Kevin\Projects\Resume_RAG_Model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install openai



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Kevin\OneDrive\Resume_RAG_Model\.venv\Scripts\python.exe -m pip install --upgrade pip


In [4]:
import sys
!{sys.executable} -m pip install openai python-dotenv

  Using cached openai-2.20.0-py3-none-any.whl.metadata (29 kB)
  Using cached jiter-0.13.0-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached openai-2.20.0-py3-none-any.whl (1.1 MB)
Using cached jiter-0.13.0-cp313-cp313-win_amd64.whl (202 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)

   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   ---------------------------------------- 3/3 [openai]




[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from openai import OpenAI

In [ ]:
%pip install beautifulsoup4 pymupdf semantic-text-splitter sentence-transformers chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install beautifulsoup4



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Kevin\OneDrive\Resume_RAG_Model\.venv\Scripts\python.exe -m pip install --upgrade pip


In [ ]:
pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#Load PDFs
def load_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [7]:
# Load HTML
def load_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
    return soup.get_text(separator='\n', strip=True) #clean text extraction

## Documents & Webpages

Here we will load in PDFs and webpages for our model


In [102]:

Kevin_resume = load_pdf('Kevin_Johnson_res.pdf')
Scratch_CNN = load_html('K3V1NJ0hnson_Scratch_Conv_Neural_Network.html')
Hurricane_project = load_html('K3V1NJ0hnson_Hurricane_Season_Predictors.html')
Capstone_project = load_html('K3V1NJ0hnson_Rental_Growth_Pre_Post_COVID.html')
Home_Loan_Approval_predictor = load_html("K3V1NJ0hnson_Home-Loan-Approval-ML-Project.html")
Customer_potential = load_html("K3V1NJ0hnson_Customer_Potential.html")

## Clean the text 

Next we will clean the text for better utilization of our RAG model


In [103]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text) # Normalize whitespace
    text = re.sub(r'Page \d+', '', text) #REmove page numbers
    return text.strip()

In [137]:

chunk_size = 300 # tweak 800/1000/etc.
splitter = TextSplitter(chunk_size)

docs = [
    ("Resume", "Kevin_Johnson_res.pdf", clean_text(Kevin_resume)),
    ("GitHub", "K3V1NJ0hnson_Scratch_Conv_Neural_Network.html", clean_text(Scratch_CNN)),
    ("GitHub", "K3V1NJ0hnson_Hurricane_Season_Predictors.html", clean_text(Hurricane_project)),
    ("GitHub", "K3V1NJ0hnson_Rental_Growth_Pre_Post_COVID.html", clean_text(Capstone_project)),
    ("GitHub", "K3V1NJ0hnson_Home-Loan-Approval-ML-Project.html", clean_text(Home_Loan_Approval_predictor)),
    ("GitHub", "K3V1NJ0hnson_Customer_Potential.html", clean_text(Customer_potential))
]

all_chunks = []
for category, source_name, text in docs:
    chunks = splitter.chunks(text)
    for i, chunk in enumerate(chunks):
        all_chunks.append(
            {
                "text": chunk,
                "metadata":{
                    "category": category,
                    "source": source_name,
                    "chunk_index": i
                }
            })
        
print(f"Total chunks: {len(all_chunks)}")
print(all_chunks[0]["metadata"])  # Print first chunk as sample


Total chunks: 120
{'category': 'Resume', 'source': 'Kevin_Johnson_res.pdf', 'chunk_index': 0}


In [138]:
# embedding model from Hugging Face.
# May change this later to something more robust
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

print("Embedding model loaded.")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1391.18it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding model loaded.


In [139]:
texts = [c["text"] for c in all_chunks]
metadatas = [c["metadata"] for c in all_chunks]

print(f"Number of chunks: {len(texts)}")
print("Example metadata:", metadatas[0])

Number of chunks: 120
Example metadata: {'category': 'Resume', 'source': 'Kevin_Johnson_res.pdf', 'chunk_index': 0}


In [140]:
embeddings = embed_model.encode(
    texts, 
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

print("Embeddings shape:", embeddings.shape)

Batches: 100%|██████████| 4/4 [00:00<00:00,  9.77it/s]

Embeddings shape: (120, 384)


In [141]:

print(type(embeddings))       # should be <class 'numpy.ndarray'>
print(embeddings[0][:10])     # first 10 dims of first chunk


<class 'numpy.ndarray'>
[-0.04944386 -0.03049594  0.09772719  0.01620435 -0.02522486  0.01099211
 -0.01570248 -0.00734629  0.01563937 -0.02282   ]


In [142]:
# Initialize Chroma client
current_dir = os.getcwd()
db_path = os.path.join(current_dir, "resume_chroma_db")

client = chromadb.PersistentClient(
    path=db_path,
    settings=Settings(anonymized_telemetry=False)
)

collection = client.get_or_create_collection("Kevin_profile")
print(f"Chroma persistent client ready at: {db_path}")

Chroma persistent client ready at: c:\Users\Kevin\Projects\Resume_RAG_Model\resume_chroma_db


In [172]:
# 1."About Me" text
about_me_text = """
### About Kevin: The Decathlete of Data

### EDUCATION AND DEGREES
- **Bachelor of Science in Data Science** (Arizona State University)
- **Bachelor of Science in Supply Chain Management** (University of Houston)

I am a dual-degree Data Scientist and former D1 scholarship athlete for the University of Houston. 
I compete in the Decathlon, with my favorite events being the 110m hurdles, discus, and high jump. 

### Key Achievements & Background
- **Elite Versatility:** I was an All-Conference athlete my freshman year and served as Team Captain. 
- **The Competitive Edge:** I once defeated an Olympic gold medalist in his respective event during a college meet.
- **Dual Degrees:** I hold a B.S. in Supply Chain Management (Univ. of Houston) and a B.S. in Data Science (Arizona State Univ.).
- **Unmatched Discipline:** Outside of tech, I've completed 10,000+ rideshare trips with a perfect 5.0-star rating.

### Professional Philosophy
I am a perfectionist who thrives on mentally stimulating challenges. I find my 'flow' when diving deep 
into complex AI and Data Science problems. While I hold my work to an incredibly high standard, 
I am passionate about building production-ready AI tools that solve real-world problems.

### Geography
Raised in Houston, TX; spent 3 years in Las Vegas and 3 years in Orange County, CA, before returning to Texas.
"""

# 2. Add it to collection
collection.add(
    ids=["about_me_01"],
    documents=[about_me_text],
    metadatas=[{"category": "About Me", "source": "Personal Bio"}]
)

print("✅ About Me section successfully added to the Vector Database!")

✅ About Me section successfully added to the Vector Database!


In [186]:
project_context = """
### PROJECT TECH SPECS: Resume RAG Assistant
This specific project is built using a RAG (Retrieval-Augmented Generation) architecture.
Kevin built this project to help recruiters understand his background and why you should give him an interview
- **Vector DB:** ChromaDB
- **Embedding Model:** sentence-transformers/all-MiniLM-L6-v2
- **LLM:** OpenAI GPT-4o-mini
- **Data Sources:** PDF Resumes, GitHub Markdown files, and manual Bio strings.
- **Features:** Custom query normalization to handle typos and semantic search to find relevant experience across multiple documents.
"""

collection.add(
    ids=["project_tech_01"],
    documents=[project_context],
    metadatas=[{"category": "Project Details", "source": "Architecture Docs"}]
)

In [187]:
# Create IDs
ids =[f"chunk-{i}" for i in range(len(texts))]

In [188]:
# Add documents to ChromaDB
collection.add(
    ids=ids,
    documents=texts,
    metadatas=metadatas,
    embeddings=embeddings.tolist()
)

print("Added chunks:", len(ids))

Added chunks: 120


In [189]:
query = "education university degree college Kevin"

q_emb = embed_model.encode(
    [query],
    convert_to_numpy=True,
    normalize_embeddings=True
)[0]

results = collection.query(
    query_embeddings=[q_emb],
    n_results=3,
)

docs = results["documents"][0]
metas = results["metadatas"][0]

for i, (doc, meta) in enumerate(zip(docs, metas), start=1):
    print(f"\n--- RESULT {i} ---")
    print(meta)
    print(doc[:500], "...")



--- RESULT 1 ---
{'category': 'Resume', 'chunk_index': 0, 'source': 'Kevin_Johnson_res.pdf'}
Kevin Johnson Austin, Texas • 832-514-9765 • kevin.johnson.data@gmail.com • LinkedIn | GitHub Education B.S. ...

--- RESULT 2 ---
{'category': 'Resume', 'source': 'Kevin_Johnson_res.pdf', 'chunk_index': 2}
Science graduate (Summa Cum Laude) with hands-on experience building machine learning and applied AI systems, including Retrieval-Augmented Generation (RAG) and computer vision models. ...

--- RESULT 3 ---
{'category': 'Resume', 'chunk_index': 1, 'source': 'Kevin_Johnson_res.pdf'}
Data Science | Arizona State University | 2023 – 2025​ - Graduated Summa Cum Laude (GPA 3.89)​ - Completed projects in ML, classification, clustering, and CNNs B.B.A. in Logistics & Supply Chain Management | University of Houston | 2009 – 2014​ - NCAA Division I Decathlete, Team Captain Profile Data ...


In [190]:
import os
from dotenv import load_dotenv

load_dotenv()

ai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI Client Ready")

✅ OpenAI Client Ready


In [191]:
def build_prompt(query,docs):
    context = "\n\n---\n\n".join(docs)

    return f"""
You are Kevin's AI Career Assistant. Your job is to help recruiters understand Kevin's 
background by answering questions using ONLY the resume context provided below.

Guidelines:
 - Be professional, enthusiastic, and concise.
 - If the answer isn't in the context, say: "I'm sorry, Kevin's resume doesn't 
   specifically mention that, but you can reach out to him directly for more details."
 - Use bullet points for lists (like skills or project tasks).
 - Always frame answers to highlight Kevin's technical growth and problem-solving.

Context from Kevin's Resume & Portfolio:
{context}

Recruiter Question:
{query}

Professional Answer:
"""

In [202]:
def answer_question(query, n_results=4):

    # Easter Egg

    if "go coogs" in query.lower() or "decathlon" in query.lower():
        return (
            "🐾 **WHOOSE HOUSE?! COOGS HOUSE!** 🐾\n\n"
            "You found the secret athlete mode! Kevin isn't just a Data Scientist; \n"
            "he's a D1 Decathlete who spent his college years hurdles-jumping \nand "
            "discus-throwing. If you're looking for a teammate with the stamina \nof a "
            "1500m runner and the explosive power of a shot-putter, you're looking at him!",
            ["Internal Easter Egg"], None, None
        )

    #Normalize query
    query_clean = query.lower().replace("kevins", "kevin's")
    
    # 1. Embed the query
    q_emb = embed_model.encode(
        [query_clean],
        convert_to_numpy=True,
        normalize_embeddings=True,
    )[0]
    
    # 2. Retrieve relevant chunks from Chroma
    results = collection.query(
        query_embeddings=[q_emb],
        n_results=n_results
    )

    docs = results["documents"][0]
    metas = results["metadatas"][0]
    distances = results["distances"][0]

    # 3. Build prompt
    prompt = build_prompt(query, docs)

    # 4. LLM call
    response = ai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are Kevin's professional Career Assistant. "
                                          "Use the provided context to answer questions about his experience."
                                          "When asked about education, always look for BOTH of his undergraduate degrees."
                                          "He has a dual-degree background; ensure you mention both ASU and the University of Houston."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=500,
        temperature=0.3,
    )
    
    answer = response.choices[0].message.content

    # 5. Build sources list
    unique_sources = []
    for meta in metas:
        source_label = meta.get("category", "General")
        unique_sources.append(source_label)
    
    unique_sources = sorted(list(set(unique_sources)))

    

    return answer, unique_sources, docs, distances



In [214]:
ans, src, d, dist = answer_question("What about Kevins skills?")
print(f"AI: {ans}")
print(f"Sources: {src}")

AI: Kevin possesses a diverse skill set that highlights his technical growth and problem-solving abilities in the field of Data Science. His key skills include:

- **Data Analysis & Visualization:** Proficient in extracting insights from complex datasets and presenting them in a clear, actionable format.
- **Machine Learning:** Experienced in developing and implementing machine learning models, including custom convolutional neural networks (CNNs).
- **Programming Languages:** Proficient in Python, R, and SQL, enabling him to tackle various data-related challenges.
- **Statistical Analysis:** Strong foundation in statistical methods, enhancing his ability to interpret data effectively.
- **Project Management:** Demonstrated ability to manage projects from conception to execution, ensuring timely delivery of high-quality results.

Kevin's combination of technical expertise and competitive spirit makes him a valuable asset in any data-driven environment.
Sources: ['About Me', 'GitHub', '

In [158]:
ans, src, docs, dist = answer_question("What is Kevin's education?")

print("--- CHUNKS SENT TO AI ---")
for i, d in enumerate(docs):
    print(f"Chunk {i+1}:\n{d}\n")

--- CHUNKS SENT TO AI ---
Chunk 1:
Kevin Johnson Austin, Texas • 832-514-9765 • kevin.johnson.data@gmail.com • LinkedIn | GitHub Education B.S.

Chunk 2:
Science graduate (Summa Cum Laude) with hands-on experience building machine learning and applied AI systems, including Retrieval-Augmented Generation (RAG) and computer vision models.

Chunk 3:
Continuous Learning: While the transfer learning models plateaued early, the custom CNN's accuracy curve was still climbing at the end of the training run, suggesting even higher potential with more

Chunk 4:
Professional Experience Machine Learning Engineer Intern | Green Rush Packaging | Fall 2025 - Built a Retrieval-Augmented Generation (RAG) model to answer state specific regulatory compliance questions from sales reps saving sales manager roughly 5 hours per week - Implemented document ingestion,

Chunk 5:
using classification models and behavioral analysis.

Chunk 6:
📊 The Dataset The project utilizes a comprehensive dataset (derived fro